In [1]:
import sys
import struct
import argparse

import numpy as np
import openhd as hd

/home/hpc/iwi3/iwi3083h/.local/lib/python3.8/site-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [2]:
!rm -rf ~/.openhd

In [3]:
Q = 10
D = 10000
hd.init(D=D, context=globals())


In [4]:
train_filename = "/home/hpc/iwi3/iwi3083h/openhd/examples/dataset/isolet_train.choir_dat"
test_filename = "/home/hpc/iwi3/iwi3083h/openhd/examples/dataset/isolet_test.choir_dat"

In [5]:
feature_matrix, labels, n_classes = hd.utils.read_choir_dat(train_filename)

In [6]:
labels

array([ 0,  0,  1, ..., 23, 24, 25], dtype=int32)

In [7]:
feature_matrix.shape

(6238, 617)

In [8]:
feature_matrix_tst, labels_tst, n_classes_tst = hd.utils.read_choir_dat(test_filename)

feature_matrix, feature_matrix_tst = \
        hd.utils.MatrixNormalizer().norm_two(
                feature_matrix, feature_matrix_tst)
N_tst = feature_matrix_tst.shape[0]

In [9]:
N_tst

1559

In [10]:
N = feature_matrix.shape[0]
F = feature_matrix.shape[1]
N, F

(6238, 617)

In [11]:
@hd.run
def create_random_bases():
    id_base = hd.draw_random_hypervector()
    level_base = hd.draw_random_hypervector()
    return id_base, level_base


@hd.run
def create_ids(F, id_base):
    id_hvs = hd.hypermatrix(F) # np.zeros(F, N) (not the empty list) 
    for f in range(F):
        id_hvs[f] = hd.permute(id_base, f)

    return id_hvs

@hd.run
def create_levels(Q, level_base):
    level_hvs = hd.hypermatrix(Q+1) # np.zeros((Q+1), N) (not the empty list)
    for q in range(Q+1):
        idx = int(q/float(Q) * D) / 2
        level_hvs[q] = hd.flip(level_base, idx)
        level_hvs[q] = hd.shuffle(level_hvs[q], 0)

    return level_hvs

In [12]:
with hd.utils.timing("Base hypervectors"):
    id_base, level_base = create_random_bases()
    id_hvs = create_ids(F, id_base)
    level_hvs = create_levels(Q, level_base)


[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [__ARG__id_base : __one__]
  <class '_ast.Assign'> :  [__ARG__level_base : __one__]

{'id_base': 'hypervec_type', 'level_base': 'hypervec_type', '__ARG__id_base': 'float*', '__ARG__level_base': 'float*', 'id_base_22b19319': <class 'float'>, 'level_base_5541502b': <class 'float'>}
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [__ARG__id_hvs : __ARG__id_base]

{'f': <class 'int'>, 'F': <class 'int'>, 'id_hvs': 'hypermat_type', 'id_base': 'hypervec_type', '__ARG__id_base': 'float*', '__ARG__id_hvs': 'float*', 'id_base_f_16a56737': <class 'float'>, 'id_hvs_f_00177a23': <class 'float'>}
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [level_hvs_q_7acb546c : __ARG__level_base]
  <class '_ast.Assign'> :  [__ARG__level_hvs : level_hvs_q_7acb546c]

{'idx': <class 'int'>, 'q': 'int', 'Q': <class 'int'>, 'D': 

In [13]:
def preprocesser(
        org_feature, cnv_feature, # Predefined argument (single feature)
        Q, level_hvs, id_hvs): # arguments passed by args
    cnv_feature = int(org_feature * Q)


def encoder(
        input_features, output_hypervector, # Predefined arguments
        Q, level_hvs, id_hvs): # arguments passed by args
    for f in range(F):
        output_hypervector += level_hvs[input_features[f]] * id_hvs[f]

In [14]:
with hd.utils.timing("Encode training"):
    hv_matrix = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = feature_matrix,
            preprocess_function = preprocesser # optional
            )

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___n_____base_n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___n_____base_n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'org_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'cnv_feature': <class 'int'>, 'Q': <class 'int'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int

In [15]:
with hd.utils.timing("Encode testing"):
    hv_matrix_tst = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = feature_matrix_tst,
            preprocess_function = preprocesser # optional
            )


[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___n_____base_n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___n_____base_n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'org_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'cnv_feature': <class 'int'>, 'Q': <class 'int'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int

## Testing the size for which encoding breaks

worked = []
for i in range(feature_matrix_tst.shape[1], 0, -1):
    print(i)
    cropped_matrix = feature_matrix_tst[:2, :i]
    worked.append(cropped_matrix.size * cropped_matrix.itemsize / (1024))# KB
    try:
        hv_matrix_tst = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = cropped_matrix,
            preprocess_function = preprocesser # optional
        )
    except:
        break
    

In [16]:
@hd.run
def single_pass(hv_matrix, labels, N, n_classes):
    class_hvs = hd.hypermatrix(n_classes)

    for idx in range(N):
        class_hvs[labels[idx]] += hv_matrix[idx]

    return class_hvs

In [17]:
with hd.utils.timing("Single pass"):
    class_hvs = single_pass(hv_matrix, labels, N, n_classes)
    class_hvs.debug_print_values()

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: N>
    <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + __ARG__hv_matrix]

{'idx': <class 'int'>, 'N': <class 'int'>, 'class_hvs': 'hypermat_type', 'labels': 'np_int_array_type', 'hv_matrix': 'hypermat_type', '__ARG__hv_matrix': 'float*', '__ARG__class_hvs': 'float*', '__ARG__labels': 'int*', '__ARG__labels__STRIDE__': 'const int', 'hv_matrix_idx_ead026f6': <class 'float'>, 'labels_idx_b9c77ce4': <class 'int'>, 'class_hvs_labels_idx_df62a0b1': <class 'float'>}
Single pass	0.4764268398284912


In [18]:
def validate(labels, pred_labels):
    n_correct = (pred_labels == labels).sum()
    n_labels = len(labels)
    print(n_correct, n_labels, n_correct / float(n_labels) * 100)


In [19]:
@hd.run
def retrain(class_hvs, hv_matrix, labels, N, n_classes):
    search_results = hd.search(class_hvs, hv_matrix)

    for idx in range(N):
        if search_results[idx] != labels[idx]:
            class_hvs[labels[idx]] += hv_matrix[idx]
            class_hvs[search_results[idx]] -= hv_matrix[idx]

    return class_hvs


In [20]:
RETRAIN_ITERATIONS = 100
SHOW_STEP_RESULT = False
for it in range(RETRAIN_ITERATIONS):
    with hd.utils.timing("Retrain itereation: %d" % it):
        class_hvs = retrain(class_hvs, hv_matrix, labels, N, n_classes)

    if SHOW_STEP_RESULT and labels_tst is not None:
        validate(labels_tst, hd.search(class_hvs, hv_matrix_tst).to_numpy())

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: N>
    <class '_ast.Assign'> :  [hv_matrix_idx_ead026f6 : __ARG__hv_matrix]
    <class '_ast.If'> : 
      then : 
        <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + hv_matrix_idx_ead026f6]
        <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + hv_matrix_idx_ead026f6]
      else : 

{'idx': <class 'int'>, 'N': <class 'int'>, 'search_results': 'np_int_array_type', 'labels': 'np_int_array_type', 'class_hvs': 'hypermat_type', 'hv_matrix': 'hypermat_type', '__ARG__search_results': 'int*', '__ARG__search_results__STRIDE__': 'const int', '__ARG__hv_matrix': 'float*', '__ARG__class_hvs': 'float*', '__ARG__labels': 'int*', '__ARG__labels__STRIDE__': 'const int', 'search_results_idx_b85d35f6': <class 'int'>, 'labels_idx_b9c77ce4': <class 'int'>, 'hv_matrix_idx_ead026f6': <class 'float'>, 'class_hvs_labels_idx_df62a0b1': <class 'float'>, 'class_hvs_search_results_idx_572d84

In [21]:
def validate(labels, pred_labels):
    n_correct = (pred_labels == labels).sum()
    n_labels = len(labels)
    print(n_correct, n_labels, n_correct / float(n_labels) * 100)


In [22]:
@hd.run
def assoc_search(class_hvs, hv_matrix_tst):
    ret = hd.search(class_hvs, hv_matrix_tst)
    return ret

with hd.utils.timing("Testing with class model\n"):
    search_results = assoc_search(class_hvs, hv_matrix_tst)

validate(labels_tst, search_results.to_numpy())

Testing with class model
	0.09038066864013672
1452 1559 93.13662604233483


In [23]:
search_results = assoc_search(class_hvs, hv_matrix)
validate(labels, search_results.to_numpy())

6238 6238 100.0


In [24]:
n_classes

26